<a href="https://colab.research.google.com/github/mingikim1/AI_software_term_project/blob/main/TermprojectB_201901001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q/A system 코드를 하단에 자유롭게 작성해주세요

TA는 하단 코드 실행 후

./release 폴더 하단에 생성된 파일을 기준으로 평가를 진행합니다
파일명은 {본인학번}.test.json으로 저장

===================== 필수 ===============================

1. ./data/test.json 파일 기준으로 코드 변경
  
  현재는 테스트용 valid.json 이 제공되었지만 test.json 으로 실행 될 수 있도록 코드 수정

2. test.json 형식 확인

  { question : 질문 내용, answer : 정답 }

3. 결과 파일 명은 {학번}.test.json 으로 통일
  
  파일은 ./realease 폴더 아래 위치

  예시) ./release/{학번}.test.json

4. 반드시 실행 여부 테스트 후에 제출, 실행 안 될 시 점수 x  

In [ ]:
!mkdir data release llm

In [ ]:
# test.json으로 바꿈, 경로만 변경 필요
!wget --no-check-certificate 'https://drive.google.com/uc?id=1ZERESR4o7Wcx7J1GsHZX9dAjPSilvmgQ&export=download' -O ./data/test.json

--2024-12-20 13:15:42--  https://drive.google.com/uc?id=1ZERESR4o7Wcx7J1GsHZX9dAjPSilvmgQ&export=download
Resolving drive.google.com (drive.google.com)... 142.251.175.102, 142.251.175.139, 142.251.175.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.175.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ZERESR4o7Wcx7J1GsHZX9dAjPSilvmgQ&export=download [following]
--2024-12-20 13:15:42--  https://drive.usercontent.google.com/download?id=1ZERESR4o7Wcx7J1GsHZX9dAjPSilvmgQ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.175.132, 2404:6800:4003:c1c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.175.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1011 [application/octet-stream]
Saving to: ‘./data/test.json’

./data/test.json    100%[===================>]    1011  --.-KB/s  

In [ ]:
import gdown
import zipfile
import os

file_id = "1CEuxDQakOvBSb4Xqy1d-aFZDK1GiYrdo"

# ZIP 파일을 다운로드할 경로
zip_file_path = '/content/pdf.zip'

# ZIP 파일 다운로드
gdown.download(f'https://drive.google.com/uc?id={file_id}', zip_file_path, quiet=False)

# 압축을 풀 폴더 경로
unzip_dir = '/content/'

# 압축 풀기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1CEuxDQakOvBSb4Xqy1d-aFZDK1GiYrdo
From (redirected): https://drive.google.com/uc?id=1CEuxDQakOvBSb4Xqy1d-aFZDK1GiYrdo&confirm=t&uuid=ffc9f253-a164-4a88-82ca-66cefabdce78
To: /content/pdf.zip
100%|██████████| 84.5M/84.5M [00:00<00:00, 93.0MB/s]


In [ ]:
!pip install accelerate bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.9 MB/s eta 0:00:00


In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)

## Load Model

In [ ]:
file_id = "1GD1Kmbzu43VoWVJOPgdyDK2UzsEUnnWY"

# ZIP 파일을 다운로드할 경로
zip_file_path = '/content/llm.zip'

# ZIP 파일 다운로드
gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_file_path, quiet=False)

# 압축을 풀 폴더 경로
unzip_dir = '/content/llm'

# 압축 풀기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "/content/llm",
    torch_dtype=torch.bfloat16,  # bfloat16
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("/content/llm")

## PDF Pre-processing

In [ ]:
!pip install langchain-community

In [ ]:
!pip install pdfminer.six

In [ ]:
import re
from langchain.document_loaders import PDFMinerLoader

# 1. PDF에서 텍스트 로드
def load_pdf_text(pdf_path):
    loader = PDFMinerLoader(pdf_path)
    docs = loader.load()
    # 모든 페이지의 텍스트를 하나로 결합
    text = " ".join([doc.page_content for doc in docs])
    return text

# 2. 불필요한 헤더 제거
def remove_header(text):
    header_pattern = r'^Open in app\n\nSearch\n\n.*?More\n\n'  # 헤더 패턴
    cleaned_text = re.sub(header_pattern, '', text, flags=re.DOTALL)  # 헤더 제거
    return cleaned_text

# 3. 페이지 구분 문자 수정
def modify_page_breaks(text):
    # 페이지 구분자 \n\n\x0c를 \n\n로 바꾸기
    text = re.sub(r'\n\n\x0c', '\n\n', text)  # 페이지 구분자 수정
    return text

# 4. 문장 중간의 개행 제거
def remove_mid_sentence_newlines(text):
    # 문장 중간의 개행 제거 (문장 끝은 제외)
    text = re.sub(r'(?<=\w)\n+(?=\w)', ' ', text)  # 단어 사이의 \n을 공백으로 대체
    return text

# 5. 문장 사이에 있는 불필요한 \n\n 제거
def remove_unwanted_newlines(text):
    # 문장 사이의 불필요한 \n\n을 공백으로 대체
    text = re.sub(r'(?<=\w)\n\n(?=\w)', ' ', text)  # 문장 사이의 \n\n을 공백으로 대체
    return text

# 6. 콤마 뒤의 불필요한 \n\n 제거
def remove_newlines_after_comma(text):
    # 콤마 뒤에 있는 \n\n을 공백으로 대체
    text = re.sub(r',\n+', ', ', text)  # 콤마 뒤에 오는 \n을 공백으로 대체
    return text

# 7. 마침표 앞의 \n\n 제거
def remove_newlines_before_period(text):
    # 문장 끝의 마침표 앞에 있는 \n\n을 공백으로 대체
    text = re.sub(r'\n\n(?=\.)', ' ', text)  # 마침표 앞의 \n\n을 공백으로 대체
    return text


# 8. \n\n으로 구분된 부분 확인
def find_newline_separated_blocks(text):
    # \n\n으로 구분된 부분 찾기
    blocks = re.split(r'\n\n', text)
    return blocks

def create_overlapping_blocks(blocks, block_size=5, overlap_size=2):
    overlapping_blocks = []

    # Start creating overlapping blocks
    for i in range(0, len(blocks), block_size - overlap_size):
        # Extract a block of the specified size
        block = blocks[i:i + block_size]

        # Join the blocks into one chunk of text
        overlapping_blocks.append(" ".join(block))

    return overlapping_blocks

all_text = []

for i in range(1, 52):
  pdf_path = f'/content/pdf/hugman_2024_ML_book-{i}.pdf'
  text = load_pdf_text(pdf_path)

  # PDF 텍스트 로드
  text = load_pdf_text(pdf_path)

  # 헤더 제거
  cleaned_text = remove_header(text)

  # 페이지 구분자 수정
  cleaned_text = modify_page_breaks(cleaned_text)

  # 문장 중간의 개행 제거
  cleaned_text = remove_mid_sentence_newlines(cleaned_text)

  # 문장 사이의 불필요한 \n\n 제거
  cleaned_text = remove_unwanted_newlines(cleaned_text)

  # 콤마 뒤의 불필요한 \n\n 제거
  cleaned_text = remove_newlines_after_comma(cleaned_text)

  # 마침표 앞의 \n\n 제거
  cleaned_text = remove_newlines_before_period(cleaned_text)

  # \n\n으로 구분된 부분 찾기
  blocks = find_newline_separated_blocks(cleaned_text)

  overlapping_blocks = create_overlapping_blocks(blocks)

  all_text.extend(overlapping_blocks)
  print("Rate: {}/51".format(i))

## Text Embedding

In [ ]:
!pip install langchain_huggingface

In [ ]:
file_id = "1Z5TpgJ_Q0tfQywU4aI4wvpJsnmAWCsss"

# ZIP 파일을 다운로드할 경로
zip_file_path = '/content/embedding.zip'

# ZIP 파일 다운로드
gdown.download(f'https://drive.google.com/uc?id={file_id}', zip_file_path, quiet=False)

# 압축을 풀 폴더 경로
unzip_dir = '/content/'

# 압축 풀기
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="/content/embedding",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

## FAISS DB

In [ ]:
!pip install faiss-cpu

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [ ]:
from langchain.schema import Document

# overlapping_blocks를 Document 객체로 변환
documents = [Document(page_content=block) for block in all_text]

# FAISS VectorStore 생성
vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)


In [ ]:
retriever = vectorstore.as_retriever()

## Prompt

In [ ]:
!pip install langchain

In [ ]:
from langchain import PromptTemplate

prompt = PromptTemplate.from_template(
    """#Instruct:
       Answer the question related to artificial intelligence knowledge.
       Use the PDF context as a helpful reference to extract relevant information, but ensure your answer integrates a balanced understanding of both the provided context and your general knowledge.
       Avoid over-reliance on the context if it does not fully address the question.
       Your answer should be strictly focused on addressing the question directly, providing the most relevant and complete response possible.
       Ensure your response is detailed enough to provide clarity, yet concise and free of unnecessary elaborations.
       Write in a natural, conversational tone while maintaining the technical accuracy required for the topic.
       **Do not include any extra sections, tags, or annotations** like #Explanation, #References, #Note, or any other similar labels.
       Provide only the final answer in a clean and natural sentence.

       #Question:
       {question}

       #Context:
       {context}

       #Answer:"""
)


## Chain

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=256,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [ ]:
# Just checking
response = chain.invoke("What is transformer?")
clean_response = response.split("#Answer:")[-1].strip()
clean_response

In [ ]:
import json

# valid.json 파일 열기
with open("/content/data/test.json", "r") as file:
    questions = json.load(file)

# 답변 생성 및 저장
output = []
for entry in questions:
    question = entry["question"]
    # 질문을 처리하고 답변 생성
    response = chain.invoke(question)
    clean_response = response.split("#Answer:")[-1].strip()  # #Answer 뒤의 내용만 추출
    # 결과 저장
    output.append({
        "question": question,
        "answer": clean_response
    })

# 결과 JSON 파일 저장
with open("/content/release/201901001.test.json", "w") as file:
    json.dump(output, file, indent=4)